In [ ]:
import pyelabtools as elab
import pandas as pd
import yaml
import json

In [ ]:
# Column assignment of the csv file with user data from the registration webpage
COLUMN_GROUP = 'Id'
COLUMN_FIRSTNAME = 'Firstname'
COLUMN_LASTNAME = 'Lastname'
COLUMN_EMAIL = 'Email'

# Further definitions
GROUPNAME_PREFIX = 'DemoLab SoSe 2024'
GROUPNAME_TUTORS = 'Tutors'

# Configuration

In [ ]:
# Server and API data for eLabFTW
API_HOST_URL = 'https://your_elabftw_url/api/v2'
# an API key with read/write permissions is required that was generated in the eLabFTW team of the lab course
API_KEY = 'your_api_key' 

# csv file with user data from the registration webpage
FILE_STUDENTDATA = 'registrations.csv'

# csv file with user data of the tutors
FILE_TUTORDATA = 'tutors.csv'

# File with definition of all lab course experiments
FILE_EXPERIMENTS = 'demolab_experiments.yaml'

# Connect to eLabFTW

In [ ]:
# Connect to eLabFTW
elab.connect(API_HOST_URL, API_KEY)

# Read and display the team that is related to the API key; this team is used for setting up the lab course structure
team = elab.get_currentteam()
print(f"Team related to the API key: {team.name} (elabftw_teamid = {team.id})")

# Create users and groups

In [ ]:
# Students:
# Read the file with user data and display the student groups
# In this step, nothing will be written into the ELN
registrationdata = pd.read_csv(FILE_STUDENTDATA, delimiter=';')

groups = {}
users = {}

# Read users and groups
for registration in registrationdata.iterrows():
    groupno = registration[1][COLUMN_GROUP]
    groupname = GROUPNAME_PREFIX + ' G' + str(groupno).zfill(3)
    groups[groupno] = {'groupname': groupname, 'members': []}
    for i in [1, 2]:
        email = registration[1][COLUMN_EMAIL+str(i)]
        if email in users.keys():
            print(f"Warning: duplicate registration of {email}")
        users[email] = {'lastname': registration[1][COLUMN_LASTNAME+str(i)], 
                        'firstname': registration[1][COLUMN_FIRSTNAME+str(i)],
                        'email': email}
        groups[groupno]['members'].append(email)
    
# Show all lab course registrations
print('Listing registrations read from file ...')
for groupno, groupdata in groups.items():
    print()
    print(f"Group number: {groupno}")
    print(f"Group name: {groupdata['groupname']}")
    for member in groupdata['members']:
        print(f"- {users[member]['lastname']}, {users[member]['firstname']}, {users[member]['email']}")

In [ ]:
# Tutors:
# Read the file with user data and display the tutors
# In this step, nothing will be written into the ELN
registrationdata = pd.read_csv(FILE_TUTORDATA, delimiter=';')

# Create tutor group
groupno = 0
groupname = GROUPNAME_PREFIX + ' ' + GROUPNAME_TUTORS
groups[groupno] = {'groupname': groupname, 'members': []}

# Read users
for registration in registrationdata.iterrows():
    email = registration[1][COLUMN_EMAIL]
    if email in users.keys():
        print(f"Warning: duplicate registration of {email}")
    users[email] = {'lastname': registration[1][COLUMN_LASTNAME], 
                    'firstname': registration[1][COLUMN_FIRSTNAME],
                    'email': email}
    groups[groupno]['members'].append(email)
    
# Show all tutors
print('Listing tutors read from file ...')
groupdata = groups[groupno]
print()
print(f"Group number: {groupno}")
print(f"Group name: {groupdata['groupname']}")
for member in groupdata['members']:
    print(f"- {users[member]['lastname']}, {users[member]['firstname']}, {users[member]['email']}")

In [ ]:
# Create users and usergroups in eLabFTW
selectedgroups = []
mode = 'all' # 'all' or 'include' or 'exclude' groups listed in selectedgroups (list of numerical group numbers)

print('Accessing eLabFTW ...')
for groupno, groupdata in groups.items():
    if (mode == 'all') or (mode == 'include' and groupno in selectedgroups) or (mode == 'exclude' and groupno not in selectedgroups):
        groupid = elab.create_group(groupdata['groupname'])  
        print()
        print(f"{groupdata['groupname']} >> elabftw_groupid = {groupid}")
        for member in groupdata['members']:
            userid = elab.create_user_in_team(users[member]['firstname'], users[member]['lastname'], users[member]['email'])
            print(f"- {users[member]['lastname']}, {users[member]['firstname']}, {users[member]['email']} >> elabftw_userid = {userid}")
            elab.add_user_to_group(groupid, userid)

print()
print('Done.')

# Create lab course days and experiments

Structure and experiments of the lab course are defined in a yaml file.

In [ ]:
# Make changes only for specific groups, lab course days or experiments

# Groups (None = all groups, [1, 2] = only groups 1 and 2)
limitgroups = None

# Lab course days (None = all lab course days, ['labcourseday_1'] = only labcourseday 1)
limitlabcoursedays = None

# Experiments (None = all experiments, ['experiment_1.1'] = only experiment 1.1)
limitexperiments = None

In [ ]:
# Load from file and display the lab course structure and experiments
# In this step, nothing will be written into the ELN

with open(FILE_EXPERIMENTS, 'r', encoding='utf8') as f:
    config = yaml.full_load(f)
    
# Show lab course structure and experiments
print("Displaying lab course structure and experiments ...")
print()

print("Categories:")
for category in config['categories']:
    print(f"- {category['name']} [color: {category['color']}{(', default' if 'default' in category.keys() and category['default'] else '')}]")
print()

print("Statuses:")
for status in config['statuses']:
    print(f"- {status['name']} [color: {status['color']}{(', default' if 'default' in status.keys() and status['default'] else '')}]")
print()

print("Templates used for experiments:")
for key, template in config['templates'].items():
    print(f"- {key}: {template['name']} [file: {template['file']}]")
print()

for labcourseday in config['labcoursedays'].values():
    print(f"{labcourseday['name']} [tag = {labcourseday['tag']}, template = {labcourseday['template']}]")
    for experiment in labcourseday['experiments'].values():
        print(f"- {experiment['name']} [template = {experiment['template']}]")

In [ ]:
# Create categories and statuses in eLabFTW

print("Accessing eLabFTW ...")
print()

print("Delete all existing categories and statuses:")
elab.delete_all_experiment_categories()
elab.delete_all_experiment_statuses()
print("Done.")
print()

print("Create categories:")
for category in config['categories']:
    name = category['name']
    color = category['color']
    default = True if 'default' in category.keys() and category['default'] else False
    print(f"- {name} [color = {color}, default = {default}]")
    elab.create_experiment_category(name, color, default)
print('Done.')
print()

print("Create statuses:")
for status in config['statuses']:
    name = status['name']
    color = status['color']
    default = True if 'default' in status.keys() and status['default'] else False
    print(f"- {name} [color = {color}, default = {default}]")
    elab.create_experiment_status(name, color, default)
print('Done.')
print()

In [ ]:
# Create templates in eLabFTW

print("Accessing eLabFTW ...")
print()

def parse_string(string: str):
    parts = string.split('###')
    for idx, part in list(enumerate(parts))[1::2]:
        key, value = part.split('=', 1)
        value = eval(f"f'{value}'")
        if key == 'group':
            id = elab.get_groupid(value)
        elif key == 'category':
            id = elab.get_experiment_categoryid(value)
        elif key == 'status':
            id = elab.get_experiment_statusid(value)
        else:
            continue
        if id is None:
            raise Exception(f'Error: {key}={value} could not be resolved')
        parts[idx] = str(id)
    return ''.join(parts)

print("Create templates:")
for key, template in config['templates'].items():
    with open(template['file'], 'r', encoding='utf8') as f:
        template_json = f.read()
    template_json = json.loads(parse_string(template_json))
    templateid = elab.create_template(template_json)
    print(f"- {key}: {template['name']} [file: {template['file']}] >> elabftw_templateid = {templateid}")
print('Done.')

In [ ]:
# Read group, user and template ids from eLabFTW and seek the groupid of the tutors user group
# The ids are required for the generation of all experiments and for assigning user rights

groups = elab.get_groups()
team = elab.get_currentteam()
print(f"User groups in the eLabFTW team '{team.name}':")

tutors_groupid = None
for group in groups:
    print(f"- {group.name} [elabftw_groupid = {group.id}]")
    if group.name == GROUPNAME_PREFIX + ' ' + GROUPNAME_TUTORS:
        tutors_groupid = group.id
print()

# Load templates from eLabFTW
print('Templates:')

templates = {}
for key, template in config['templates'].items():
    templates[key] = elab.get_templateid(template['name'])
    print(f"- {key}: {template['name']} [elabftw_templateid = {templates[key]}]")

In [ ]:
# Create lab course structure and experiments in eLabFTW

# Helper function that creates an experiment and fills in the metadata
def create_and_configure_experiment(templateid, title, tags, groups_readaccess, groups_writeaccess, groupnumber, users):
    # Create experiment
    eid = elab.create_experiment_from_template(templateid=templateid,
                                               title=title,
                                               tags=tags,
                                               groups_readaccess=groups_readaccess,
                                               groups_writeaccess=groups_writeaccess)
    # Add group number to metadata
    elab.update_experiment_metadata(eid, 'Group number', groupnumber)
    # Link users as group members in the metadata
    for idx in range(len(users)):
        elab.update_experiment_metadata(eid, 'Student '+str(idx+1), users[idx].userid)
    return eid

# Create experiments for all groups (in reverse order for meaningful presentation in to-do list)
keyword_studentgroup = GROUPNAME_PREFIX + ' G'
for group in groups:
    if group.name.startswith(keyword_studentgroup):
        groupnumber = int(group.name[len(keyword_studentgroup)+1:])
        if limitgroups is None or groupnumber in limitgroups:
            print(group.name)
            grouptag = group.name.replace(" ", "_")
            for pkey, labcourseday in reversed(config['labcoursedays'].items()):
                if limitlabcoursedays is None or pkey in limitlabcoursedays:
                    print(f"- {labcourseday['name']}")
                    eids = []
                    for ekey, experiment in reversed(labcourseday['experiments'].items()):
                        if limitexperiments is None or ekey in limitexperiments:
                            eid = create_and_configure_experiment(templateid=templates[experiment['template']],
                                                                  title=experiment['name'],
                                                                  tags=[labcourseday['tag'], grouptag],
                                                                  groups_readaccess=[group.id, tutors_groupid],
                                                                  groups_writeaccess=[group.id],
                                                                  groupnumber=groupnumber,
                                                                  users=group.users)
                            eids.append(eid)
                            print('-- ' + experiment['name'] + ' => Done.')
                    pid = create_and_configure_experiment(templateid=templates[labcourseday['template']],
                                                          title=labcourseday['name'],
                                                          tags=[labcourseday['tag'], grouptag],
                                                          groups_readaccess=[group.id, tutors_groupid],
                                                          groups_writeaccess=[group.id],
                                                          groupnumber=groupnumber,
                                                          users=group.users)
                    for eid in reversed(eids):
                        elab.link_experiment_to_experiment(pid, eid)
                    print('=> Done.')
            print()
print('Done.')

# Other functions

## List experiments

In [ ]:
# Show all experiments matching a query string
query = ''
experiments = elab.get_experiments(query)
for experiment in experiments:
    print(f"{(str(experiment.id))}: {experiment.title}")

## Delete experiments

!!! ATTENTION !!!

In [ ]:
# ATTENTION!!!! 
# By executing this cell, all experiments retrieved in the previous cell (i.e. in the variable experiments) will be deleted!

for experiment in experiments:
    elab.delete_experiment(experiment.id)
    print(f"Deleted => {(str(experiment.id))}: {experiment.title}")
print('Done.')

## List all users in the team

In [ ]:
# Show all users in the team
teamid = elab.get_currentteam().id
users = elab.get_users()

teamusers = []
for user in users:
    try:
        user = elab.get_user(user.userid)
        if teamid in [x.id for x in user.teams]:
            teamusers.append(user)
            print(f'{user.userid}: {user.fullname}, {user.firstname}')
    except:
        pass
    
print()
print(f'Done. Found {len(teamusers)} users in team.')

In [ ]:
# Display the users retrieved in the cell above (i.e. in the variable teamusers) in alphabetical order
teamusers = sorted(teamusers, key=lambda x:x.lastname)
for user in teamusers:
    print(f'{user.lastname}, {user.firstname}')

## List all groups in the team

In [ ]:
elab.get_groups()